In [1]:
#导入所需要的模块
import numpy as np
import pandas as pd
import re #用于特殊符号清洗（正则表达式）
import string #处理字符
import opencc #用于繁体字简化
import jieba #用于中文分词

In [2]:
#导入数据集
df = pd.read_csv("datasets/weibo_senti_100k.csv")
#查看数据集
df

,label,review
0,1,﻿更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你]
1,1,@张晓鹏jonathan 土耳其的事要认真对待[哈哈]，否则直接开除。@丁丁看世界 很是细心...
2,1,姑娘都羡慕你呢…还有招财猫高兴……//@爱在蔓延-JC:[哈哈]小学徒一枚，等着明天见您呢/...
3,1,美~~~~~[爱你]
4,1,梦想有多大，舞台就有多大![鼓掌]
...,...,...
119983,0,一公里不到，县医院那个天桥下右拐200米就到了！//@谢礼恒: 我靠。这个太霸道了！离224...
119984,0,今天真冷啊，难道又要穿棉袄了[晕]？今年的春天真的是百变莫测啊[抓狂]
119985,0,最近几天就没停止过！！！[伤心]
119986,0,//@毒药女流氓:[怒] 很惨!


In [3]:
#随机查看5条数据
df.sample(5)

,label,review
103009,0,[吃惊][怒]神马世道！妇女儿童请保护自己，小心，小心！//@幸福育儿圣经: 大家要小心！[...
71517,0,好看的餐桌，我家什么时候才能有？！[泪]身不由己啊！
28186,1,真的好坏，抢作业本，哈哈//@姚晨：坏蛋好坏啊！[嘻嘻]
37022,1,//@宁波慈城古县城:[哈哈][话筒]
1727,1,[哈哈][哈哈] //@小summerkeno:转发微博


In [4]:
#查看数据集简要信息
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119988 entries, 0 to 119987
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   119988 non-null  int64 
 1   review  119988 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.8+ MB


In [5]:
#数据预处理
#检查是否存在缺失值
df.isna().sum()

label     0
review    0
dtype: int64

In [6]:
#检查是否存在重复值（review列）
df['review'].duplicated().sum()

1534

In [7]:
#删除重复值
df = df.drop_duplicates(subset='review')

In [8]:
#查看数据集形状
df.shape

(118454, 2)

In [9]:
#查看情感极性分布情况(0-负向；1-正向)
print('评论数目（总体）：%d' % df.shape[0])
print('评论数目（正向）：%d' % df[df.label==1].shape[0])
print('评论数目（负向）：%d' % df[df.label==0].shape[0])

评论数目（总体）：118454
评论数目（正向）：59993
评论数目（负向）：58461


In [10]:
#中文文本预处理
#加载停用词表
stop_words = open('datasets/stopwords.txt',encoding='utf-8').readlines()
stop_words = [line.strip() for line in stop_words] #去掉换行符
#增加停用词
stop_words.append("转发微博")
stop_words.append("微博")
stop_words.append("转发")
stop_words.append("回复")
#防止被strip过滤掉
stop_words.append(" ") 
stop_words.append("\n")

In [11]:
#定义中文文本预处理函数
def chinese_pre(text):
    ##（1）清洗特殊符号
    # 去除正文中的@和回复/转发中的用户名
    text = re.sub(r"(//)?\s*@\S*?\s*(:| |$)", " ", text)  
    # 去除表情符号（[表情符号文本]）
    text = re.sub(r"\[.*?\]", "", text) 
    # 去除网址
    URL_REGEX = re.compile(
        r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))*(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’])*)',
        re.IGNORECASE)
    text = re.sub(URL_REGEX, " ", text) 
    #去除非汉字和字母的字符
    text = re.sub(r"[^\u4e00-\u9fa5a-zA-Z]+", " ", text)
     # 合并过多无意义的空格
    text = re.sub(r"\s+", " ", text)
    #（2）字母小写
    text = text.lower()
    #（3）繁体字简化
    text = opencc.OpenCC('t2s').convert(text)
    #（4）jieba中文分词
    text = list(jieba.cut(text))
    #（5）去停用词
    text = [word.strip() for word in text if word not in stop_words]
    #处理后的分词用空格连接为字符串
    text = ' '.join(text)
    return text

In [12]:
#对微博评论文本应用中文文本预处理函数
df['cutword'] = df.review.map(lambda x:chinese_pre(x))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\18417\AppData\Local\Temp\jieba.cache
Loading model cost 0.331 seconds.
Prefix dict has been built successfully.
C:\Users\18417\AppData\Local\Temp\ipykernel_20340\4078211516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cutword'] = df.review.map(lambda x:chinese_pre(x))


In [13]:
#查看分词结果是否存在缺失值（空字符串）
df.cutword.isin(['']).sum()

4800

In [14]:
#缺失值处理——删除cutword为空字符串的行
df = df[~df.cutword.isin([''])]
df.shape

(113654, 3)

In [15]:
df['cutword']

0                                 更博 爆照 帅 越来越 爱 生快 傻 缺
1                                 土耳其 事要 认真对待 开除 细心 酒店
2                             姑娘 羡慕 招财猫 学徒 一枚 明天 大佬 范儿
3                                                    美
4                                                梦想 舞台
                              ...                     
119983    公里 不到 县 医院 天桥 下右 拐 米 太 霸道 远 点 真心 找 吃 下次 记到 吃
119984                             真冷 穿 棉袄 春天 真的 百变 莫测
119985                                           几天 停止
119986                                              很惨
119987                          搞 乜 鬼 想知 入去 gotrip 睇 睇
Name: cutword, Length: 113654, dtype: object

In [16]:
#将预处理后的结果保存为新文件
df[["label","cutword"]].to_csv("datasets/label_cutword.csv", index=False)

In [17]:
#查看情感极性分布情况(0-负向；1-正向)
print('评论数目（总体）：%d' % df.shape[0])
print('评论数目（正向）：%d' % df[df.label==1].shape[0])
print('评论数目（负向）：%d' % df[df.label==0].shape[0])

评论数目（总体）：113654
评论数目（正向）：57426
评论数目（负向）：56228
